# 맥주 추천 시스템         
             
**[데이터](https://github.com/ghgit1798/Crawling-Preprocessing/tree/main/CBF_Beer)**            
             
**초기 구상**                 
맥주의 특징을 사용자가 선택하면 거기에 맞는 맥주를 추천               

### EDA / 전처리      
        
이미 최종 처리된 데이터를 사용해서 더이상의 전처리는 필요 없다고 생각      
맥주의 특징만 활용할 생각            
           
1. 파싱 (여러문자열 구조의 값을 벡터화하기 쉽게 파싱)          
2. 수동 매핑 (맥주 특징 중 같은 의미를 갖는 특징 매핑 : 예. 초콜릿 -> 초콜렛)                        
3. 수동 매핑 적용        
4. 벡터화                      

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import openpyxl

from ast import literal_eval                                  # 문자열 파싱 라이브러리
from sklearn.feature_extraction.text import CountVectorizer   # 
from sklearn.metrics.pairwise import cosine_similarity        # 코사인 유사도 라이브러리

In [2]:
df = pd.read_csv('/root/Beer/맥주_cbf_data.csv', encoding='utf-8')

df.head(1)

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body,평점평균,평가횟수
0,0,0,0,Kloud Original Gravity,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '빵 향', '달콤한 향', '풀잎 향']","['쓴 맛', '상쾌함', '생강 힌트']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '탄산']",2.30101,99


In [3]:
df = df.drop(columns = ['Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1'])

df.head(1)

,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body,평점평균,평가횟수
0,Kloud Original Gravity,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '빵 향', '달콤한 향', '풀잎 향']","['쓴 맛', '상쾌함', '생강 힌트']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '탄산']",2.30101,99


In [4]:
import importlib
import utils
importlib.reload(utils)

from utils import parse_string_list_column 

list_cols = ['Aroma', 'Flavor', 'Balance', 'Season', 'Paring Food','Body']

df = parse_string_list_column(df, list_cols)

In [5]:
df.head(1)

,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body,평점평균,평가횟수,Aroma_parsed,Flavor_parsed,Balance_parsed,Season_parsed,Paring Food_parsed,Body_parsed
0,Kloud Original Gravity,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '빵 향', '달콤한 향', '풀잎 향']","['쓴 맛', '상쾌함', '생강 힌트']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '탄산']",2.30101,99,"[고소한 맥아향, 강한 향, 빵 향, 달콤한 향, 풀잎 향]","[쓴 맛, 상쾌함, 생강 힌트]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움, 탄산]"


In [6]:
from itertools import chain

# aroma_parsed 유사값 있는지 살펴보기
all_aromas = list(chain.from_iterable(df['Aroma_parsed']))
unique_aromas = sorted(set(all_aromas))

for aroma in unique_aromas:
    print(aroma)

가벼운 곡물향
가벼운 낱알 몰트 향
가벼운 스컹스 향
가벼운 향
강한 향
거친 맥아향
고소한 맥아향
고수
과일향
깔끔한 맥아향
꽃 향기
꽃향기
노블 홉향
단맛
달콤한 맥아향
달콤한 향
라즈베리향
레몬향
로스팅 보리향
매운 향
맥아 로스팅향
맥아향
멕아 로스팅향
민트향
밀 향
바나나 향
바나나향
볶은 맥아향
북미 홉향
빵 향
사츠홉향
소나무향
순한 고수향
스컹크 향
스파이시한 향
스파이시한 효모향
쌀 향
약간의 과일향
약한 과일향
약한 빵 향
약한 스컹스 향
약한 스컹크 향
약한 캐라멜 멕아향
약한 향
오렌지
옥수수 향
은은한 맥아향
자몽향
창백한 맥아향
초콜렛 향
카라멜 향
카라멜향
캐러멜 향
캐러멜향
커피향
쿠키향
풀잎 향
풀잎향
풀향
플로랄 홉향
허브향
홉 아로마 거의 없음
홉향
후추향
희미한 옥수수향


In [7]:
# 아로마 파싱한 것 수동 매핑
aroma_mapping = {
    '매운 향':'스파이시한 향',
    '멕아 로스팅향':'맥아 로스팅향',
    '바나나향':'바나나 향',
    '약간의 과일향':'약한 과일향',
    '카라멜 향':'캐러멜 향',
    '카라멜향':'캐러멜 향',
    '캐러멜향':'캐러멜 향',
    '풀잎향':'풀잎 향',
    '풀향':'풀잎 향'}

In [8]:
# 아로마 매핑 적용
from utils import normalize_list_column

df['Aroma_parsed'] = normalize_list_column(df, 'Aroma_parsed', aroma_mapping)

In [9]:
# 모듈 import가 안 돼서 적음

import importlib
import utils
importlib.reload(utils)

from utils import vectorize_multilabel_column

In [10]:
# 아로마 벡터화
# from utils import vectorize_multilabel_column

aroma_df, aroma_mlb = vectorize_multilabel_column(df, 'Aroma_parsed', 'Aroma')

In [11]:
# flavor 유사값 살펴보기 

all_flavors = list(chain.from_iterable(df['Flavor_parsed']))
unique_flavors = sorted(set(all_flavors))

for flavor in unique_flavors:
    print(flavor)

(['청량함
)
가벼운 곡물 풍미
가벼운 맥아 풍미
가벼움
강렬함
강한 쓴맛
건포도
견고한 쓴맛
견과류 맥아 풍미
견과류 풍미
고도의 탄산
과일 풍미
구운 풍미
김치에 좋은 맥주
깔끔한 맥아 풍미
깔끔한 쓴맛
깔끔함
낮은 알코올
단 맛
단맛
단맛 거의 없음
달콤한 맥아
달콤한 쿠기 풍미
달콤함
데킬라맛
드라이한 크림 풍미
로스팅 보리향
로스팅과 구운 풍미
매끄러운 맥아 풍미
맥아 풍미
밀크세이크 질감
보조 맥아 힌트
부드러움
빵 껍질
사과 주스 맛
사과맛
사츠 홉의 풍미
상쾌함
상큼함
생강 힌트
섬세한
섬세한 맥아 풍미
섬세함
순함
스컹크 풍미
스파이시
시큼한 끝맛
시큼한 풍미
신맛
신선한 홉 풍미
쌀 풍미
쌉쌀한 끝맛
쌉쌀한 풍미
쓴 맛
쓴 맛']
약간 단맛
약간 쌉쌀함
약간 크리미
약간의 단맛
약간의 쓴 맛
약간의 쓴맛
약한 단맛
약한 달콤한 맥아
약한 떫은 맛
약한 쓴맛
약한 캐러멜
약한 홉 풍미
역겨움
옅은 맥아향
옅은 옥수수 향
옥수수 단맛
옥수수 맛
옥수수 풍미
청량한 끝맛
청량함
초콜렛
초콜릿
캐러멜
커피
크리미한 맥아 풍미
크리미함
탄산수
할러타우 홉
허브 풍미
홉 풍미 거의 없음
홉의 풍미 많지 않음


In [12]:
# 맛에서 ) 항목 제거
def clean_special_characters(df, column, chars_to_remove):
    """
    리스트 안 문자열에서 지정한 문자를 모두 제거
    """
    def clean_list(item_list):
        return [i.translate(str.maketrans('', '', chars_to_remove)).strip() for i in item_list if i.strip()]
    
    return df[column].apply(clean_list)


# ) 문자 제거
df['Flavor_parsed'] = clean_special_characters(df, 'Flavor_parsed', ')')


In [13]:
# 맛 파싱 수동 매핑

flavor_mapping = {
    "(['청량함" : '청량함',
    '단 맛':'단맛',
    '달콤한 쿠기 풍미':'달콤한 쿠키 풍미',
    '사과 주스 맛':'사과맛',
    '섬세한':'섬세함',
    "쓴 맛']":'쓴 맛',
    '약간의 쓴 맛':'약간의 쓴맛',
    '약한 쓴맛':'약간의 쓴맛',
    '옥수수 맛':'옥수수 풍미',
    '초콜릿':'초콜렛'
}

In [14]:
# flavor 매핑 적용

df['Flavor_parsed'] = normalize_list_column(df, 'Flavor_parsed', flavor_mapping)

In [15]:
# balance 유사값

all_balance = list(chain.from_iterable(df['Balance_parsed']))
unique_balance = sorted(set(all_balance))

for balance in unique_balance:
    print(balance)

곡물 풍미
균등
균형
깔끔한 끝맛
깨끗한 끝맛
다양한 밸런스를 가진 제품 존재
다양한 정도의 로스팅 풍미
단 맛
드라이
드라이한 끝맛
리치
매우 달콤
매우 드라이
맥아 풍미
미디엄
밀크셰이크 질감
부드러움
비터
상쾌함
쌉쌀한 끝맛
약간 달콤
약간 시큼함
일정 수준의 신맛
청럄함
청량함
크리미
홉에 의한 밸런스 거의 없음
홉의 풍미


In [16]:
balance_mapping = {
    '균등':'균형',
    '청럄함':'청량함'
}

In [17]:
# balance 매핑 적용

df['Balance_parsed'] = normalize_list_column(df, 'Balance_parsed', balance_mapping)

In [18]:
# season 유사값

all_season = list(chain.from_iterable(df['Season_parsed']))
unique_season = sorted(set(all_season))

for season in unique_season:
    print(season)

10월
9월
견과류
기름진 음식
더운 여름
따듯한 날씨
따뜻한 날씨
마른안주
봄
서늘한 계절
여름
연중 내내
연중내내
파자


In [19]:
season_mapping = {
    '따듯한 날씨':'따뜻한 날씨',
    '연중내내':'연중 내내'
}

In [20]:
# season 매핑 적용

df['Season_parsed'] = normalize_list_column(df, 'Season_parsed', season_mapping)

In [21]:
# paring food_parsed 유사값

all_food = list(chain.from_iterable(df['Paring Food_parsed']))
unique_food = sorted(set(all_food))

for food in unique_food:
    print(food)

가벼운 디저트
가벼운 음식
고구마 튀김
고기
고기 파이
광범위한 음식
구운 붉은 고기
구운 음식
굴
기름기 많은 생선
기름진 음식
나쵸
내장
다양한 음식
달걀
닭
닭고기
돼지고기
돼지고기 구이
든든한 음식
딤섬
떡볶이
라이트한 치즈
리치한 음식
마일드 치즈
매운 요리
매운 음식
맥앤치즈
멕시코 요리
무거운 음식
바나나 케이크
바베큐
바베큐 케이준
바비큐 고기
바이스부르스트
버거
버섯
베이컨
브라트부르스트
블루치즈
빵
살코기 생선
샌드위치
샐러드
생선 요리
소시지
스테이크
아시아 음식
아키도리
알싸한 쓴 음식
애플 파이
야키도리
어니언 링
어니언링
에그 베네딕트
여러 광범위한 음식
연어
연어 구이
영국 치즈
오리 구이
오믈렛
저크 치킨
저크 피시
체다 치즈
체다치즈
초콜릿
초콜릿칩 쿠키
커리
케이준 생선
퀘사디아
퀘사디야
크림 리조또
타코
타파스
태국 음식
토마토 파스타
톡 쏘는 치즈
파타타스 바라바스
풀드 포크
프렌치프라이
피자
필라프
한국 요리
해산물
해산물 요리
햄
햄버거
홍합
화히타
후추
훈제 생선
훈제된 음식
흰색 육류


In [22]:
food_mapping = {
    '닭':'닭고기',
    '매운 요리':'매운 음식',
    '바베큐 고기':'바베큐',
    '브라트부르스트':'바이스부르스트',
    '살코기 생선':'생선 요리',
    '아키도리':'야키도리',
    '어니언 링':'어니언링',
    '퀘사디야':'퀘사디아',
    '체다 치즈':'체다치즈',
    '해산물 요리':'해산물',
    '버거':'햄버거'
}

In [23]:
# food 매핑 적용

df['Paring Food_parsed'] = normalize_list_column(df, 'Paring Food_parsed', food_mapping)

In [24]:
# body 유사값 살펴보기

all_body = list(chain.from_iterable(df['Body_parsed']))
unique_body = sorted(set(all_body))

for body in unique_body:
    print(body)

가벼운 탄산
가벼운 탄산화
가벼움
걸쭉
깔끔함
드라이
라이트
많은 탄산
매우 크리미
미디엄
미디움
부드러움
불쾌한 탄산화
상쾌함
약간의 매끄러움
적당한 탄산
적당한 탄산화
적은 탄산
적절한 탄산화
진한 맥아 풍미
청량함
크리미
탄산
풍부한 탄산


In [25]:
body_mapping = {
    '미디엄':'미디움',
    '적당한 탄산화':'적당한 탄산',
    '적절한 탄산화':'적당한 탄산'
}

In [26]:
# body 매핑 적용

df['Body_parsed'] = normalize_list_column(df, 'Body_parsed', body_mapping)

In [27]:
flavor_df, flavor_mlb = vectorize_multilabel_column(df, 'Flavor_parsed', 'Flavor')
balance_df, balance_mlb = vectorize_multilabel_column(df, 'Balance_parsed', 'Balance')
season_df, season_mlb = vectorize_multilabel_column(df, 'Season_parsed', 'Season')
food_df, food_mlb = vectorize_multilabel_column(df, 'Paring Food_parsed', 'Paring Food')
body_df, body_mlb = vectorize_multilabel_column(df, 'Body_parsed', 'Body')

In [28]:
df.head(1)

,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body,평점평균,평가횟수,Aroma_parsed,Flavor_parsed,Balance_parsed,Season_parsed,Paring Food_parsed,Body_parsed
0,Kloud Original Gravity,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '빵 향', '달콤한 향', '풀잎 향']","['쓴 맛', '상쾌함', '생강 힌트']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '탄산']",2.30101,99,"[고소한 맥아향, 강한 향, 빵 향, 달콤한 향, 풀잎 향]","[쓴 맛, 상쾌함, 생강 힌트]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움, 탄산]"


In [29]:
features_df = pd.concat([aroma_df, flavor_df, balance_df, season_df, food_df, body_df], axis=1)

In [30]:
features_df.head(1)

,Aroma_가벼운 곡물향,Aroma_가벼운 낱알 몰트 향,Aroma_가벼운 스컹스 향,Aroma_가벼운 향,Aroma_강한 향,Aroma_거친 맥아향,Aroma_고소한 맥아향,Aroma_고수,Aroma_과일향,Aroma_깔끔한 맥아향,...,Body_불쾌한 탄산화,Body_상쾌함,Body_약간의 매끄러움,Body_적당한 탄산,Body_적은 탄산,Body_진한 맥아 풍미,Body_청량함,Body_크리미,Body_탄산,Body_풍부한 탄산
0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [31]:
print("Aroma:", aroma_df.shape[1])
print("Flavor:", flavor_df.shape[1])
print("Body:", body_df.shape[1])
print("Food:", food_df.shape[1])
print('Balance:', balance_df.shape[1])
print('Season:', season_df.shape[1])

Aroma: 56
Flavor: 77
Body: 21
Food: 83
Balance: 26
Season: 12


**벡터화까지 마친 후**            
          
초기 구상 : 벡터화까지 진행 후 분류모델을 이용해 추천          
         
**문제점** 
                    
분류 모델은 정답파일 즉, 사용자 취향 정보가 담긴 파일이 있어야 학습 가능       
           
**해결..?**  
                    
코사인 유사도 진행               

In [39]:
# 모든 벡터를 하나로 합치기..?

from functools import reduce

features_v = [aroma_df, flavor_df, body_df, food_df, balance_df, season_df]
X = reduce(lambda left, right: pd.concat([left, right], axis=1), features_v)

In [33]:
# 코사인 유사도 계산

from sklearn.metrics.pairwise import cosine_similarity

cos_sim_matrix = cosine_similarity(X)

In [36]:
# 추천 함수

def recommend_beers(index, df, sim_matrix, top_n=5):
    sim_scores = list(enumerate(sim_matrix[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # 자기 자신 제외

    beer_indices = [i for i, _ in sim_scores]
    return df.iloc[beer_indices][['맥주이름']]


In [40]:
df.iloc[10]

맥주이름                                      Bintang Pilsener
맥주스타일                 Pale Lager - International / Premium
Main Category                                           라거
Aroma                         ['고소한 맥아향', '강한 향', '가벼운 향']
Flavor                         ['쓴 맛', '상쾌함', '약한 달콤한 맥아']
Balance                                            ['미디엄']
Season                                           ['연중 내내']
Paring Food                        ['생선 요리', '해산물', '샐러드']
Body                                 ['드라이', '깔끔함', '가벼움']
평점평균                                              2.032957
평가횟수                                                   531
Aroma_parsed                        [고소한 맥아향, 강한 향, 가벼운 향]
Flavor_parsed                        [쓴 맛, 상쾌함, 약한 달콤한 맥아]
Balance_parsed                                       [미디엄]
Season_parsed                                      [연중 내내]
Paring Food_parsed                       [생선 요리, 해산물, 샐러드]
Body_parsed                                [드라이, 깔끔함, 가벼

In [37]:
recommend_beers(10, df, cos_sim_matrix, top_n=5)

,맥주이름
15,Bali Hai Premium Lager
30,Budweiser
5,Sapporo Premium Beer
17,Bavaria Pilsener
24,Kloud Fitz Super Clear
